In [15]:
%pylab

from imf_components import (
    imf_loglikelihood,
    chabrier_smooth_imf,
    chabrier_imf,
    chabrier_smooth_default_params,
    chabrier_default_params,
    generate_samples_from_imf,
)
import emcee
from multiprocessing import Pool

# masses = np.load("/home/mgrudic/kroupa_m300_samples.npy")[:10**5] #generate_samples_from_imf(10**6, chabrier_imf, (np.log10(0.2),0.55,-1,-1.3,np.log10(120.))) #np.load("/home/mgrudic/kroupa_m300_samples.npy")[:10**4] #np.loadtxt("M2e4_R1_Z0.01_S0_A2_B0.1_I10000_Res271_n2_sol0.5_42/output/IMF.dat")[:,1]
# masses = generate_samples_from_imf(10**6, chabrier_imf, (np.log10(0.2),0.55,-1,-1.3,-np.inf, np.log10(120.)))

# logm0, sigma, mbreak, alpha, mmax = params
# masses = generate_samples_from_imf(10**4, chabrier_smooth_imf,chabrier_smooth_default_params)
masses = np.loadtxt("M2e4_R1_Z0.01_S0_A2_B0.1_I10000_Res271_n2_sol0.5_42/output/IMF.dat")[:, 1]
# masses = masses[masses>0.1]
p0 = list(chabrier_smooth_default_params)
# p0[-2] = np.log10(masses.min()*0.5)
# p0[-1]= np.log10(masses.max()*2.)

from scipy.optimize import minimize


def lossfunc(x):
    return -imf_loglikelihood(x, masses, model="chabrier_smooth")


# plt.plot(np.log10(mgrid) , imf); plt.yscale('log')
# plt.ylim(imf.max()/10**4,2*imf.max())
# plt.show()

Using matplotlib backend: tkagg
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/home/mgrudic/miniconda3/lib/python3.12/site-packages/IPython/core/magics/pylab.py:166: UserWarning: pylab import has clobbered these variables: ['mgrid', 'ndim']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [80]:
p0 = list(chabrier_smooth_default_params)
params = minimize(lossfunc, p0, method="Nelder-Mead").x
print(params)
mgrid = np.logspace(-3, 3, 100001)
logm = np.log10(mgrid)
imf = chabrier_smooth_imf(logm, params)
counts, bins = plt.hist(np.log10(masses), 31)[:2]
plt.plot(np.log10(mgrid), imf * (bins.max() - bins.min()) / len(bins) * len(masses))
plt.yscale("log")
# plt.yscale('log')

[-0.32667251 -0.89096981 -0.75695373]


In [2]:
p0 = list(chabrier_smooth_default_params)
params = minimize(lossfunc, p0, method="Nelder-Mead").x
likelihood = minimize(lossfunc, p0, method="Nelder-Mead").x
params1 = np.concatenate([params, [2.5, -3.0, 3.0, -3]])


def lossfunc2(x):
    return -imf_loglikelihood(x, masses, model="chabrier_smooth_lognormal")


# print(lossfunc2(params))
params2 = minimize(lossfunc2, params1, method="Nelder-Mead").x

print(lossfunc2(params1))

l2 = lossfunc2(params2)
params_null = np.concatenate([params, [np.log10(masses.max()), -30.0, 3.0, -3]])
l1 = lossfunc2(params_null)
likelihood_ratio = l1 - l2
AIC1 = 2 * 3 - 2 * l1
AIC2 = 2 * 7 - 2 * l2
delta_AIC = AIC2 - AIC1
print(delta_AIC)
mgrid = np.logspace(-3, 4, 100001)
logm = np.log10(mgrid)
from imf_components import chabrier_smooth_lognormal_peak_imf

counts, bins = plt.hist(np.log10(masses), 31, histtype="step", edgecolor="black")[:2]
imf_to_bins = (bins.max() - bins.min()) / len(bins) * len(masses)
imf = chabrier_smooth_lognormal_peak_imf(logm, params_null)
plt.plot(np.log10(mgrid), imf * imf_to_bins, color="black", ls="dotted", label="Chabrier-like")
plt.yscale("log")
imf = chabrier_smooth_lognormal_peak_imf(logm, params2)
plt.plot(np.log10(mgrid), imf * imf_to_bins, ls="dashed", color="red", label="Chabrier-like + Lognormal")
plt.yscale("log")

plt.xlabel(r"$\log M_{\rm \star}$")
plt.ylabel(r"$N$")
plt.ylim(0.1, 5e2)
plt.xlim(-2, 3.5)
plt.legend(loc=1)
# print(likelihood_ratio)

1547.449838346706
252.99146700738856


In [17]:
print(params2)


def lnprob(x):
    params0 = x[:-4]
    log_mmax1, log_fpeak, logm0_peak, logsigma_peak = x[-4:]
    if log_mmax1 > np.log10(masses.max()):
        return -np.inf
    if np.abs(log_fpeak) > 50.0:
        return -np.inf
    if logm0_peak < 0.0 or logm0_peak > 4.0:
        return -np.inf
    if logsigma_peak > 0.0 or logsigma_peak < -4.0:
        return -np.inf

    return imf_loglikelihood(x, masses, model="chabrier_smooth_lognormal")


p0 = params2  # list(chabrier_smooth_default_params)
# p0 = minimize(lossfunc, p0,method="Nelder-Mead").x

nwalkers, ndim = 100, 7
p0 = np.array(p0) * np.exp(
    0.01 * np.random.normal(size=(nwalkers, ndim))
)  # + 0.01*np.random.normal(size=(nwalkers,ndim))
from multiprocessing import Pool

sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob)  # , pool=Pool())
state = sampler.run_mcmc(p0, 1000)
sampler.reset()
sampler.run_mcmc(state, 10000)
flat_samples = sampler.get_chain(flat=True, thin=1000)
np.save("chabrier_lognormal_samples.npy", flat_samples)
# fig = corner.corner(flat_samples) #, labels=labels#, truths=[m_true, b_true, np.log(f_true)]

[-0.30240007 -0.81797266 -0.73101858  2.06648386 -5.10399964  3.08024542
 -1.97915605]


In [18]:
def lnprob(x):
    return imf_loglikelihood(x, masses, model="chabrier_smooth")


p0 = list(chabrier_smooth_default_params)
p0 = minimize(lossfunc, p0, method="Nelder-Mead").x
print(p0)

nwalkers, ndim = 100, 3
p0 = np.array(p0) * np.exp(
    0.01 * np.random.normal(size=(nwalkers, ndim))
)  # + 0.01*np.random.normal(size=(nwalkers,ndim))
from multiprocessing import Pool

sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob)  # , pool=Pool())
state = sampler.run_mcmc(p0, 1000)
sampler.reset()
sampler.run_mcmc(state, 10000)
flat_samples = sampler.get_chain(flat=True, thin=1000)
np.save("chabrier_samples.npy", flat_samples)

[-0.32667251 -0.89096981 -0.75695373]


In [85]:
# sampler.get_autocorr_time()
# import corner

# plt.hist(flat_samples[:,-])
# plt.xlabel(r"$\log M_{\rm peak}$")
# flat_samples.shape
# fig = corner.corner(flat_samples,labels=[r"$\log m_{\rm c}$", r)  #, labels=labels#, truths=[m_true, b_true, np.log(f_true)]

In [23]:
plt.clf()
p0 = list(chabrier_smooth_default_params)
params = minimize(lossfunc, p0, method="Nelder-Mead").x
likelihood = minimize(lossfunc, p0, method="Nelder-Mead").x
params1 = np.concatenate([params, [2.5, -3.0, 3.0, -3]])


def lossfunc2(x):
    return -imf_loglikelihood(x, masses, model="chabrier_smooth_lognormal")


# print(lossfunc2(params))
params2 = minimize(lossfunc2, params1, method="Nelder-Mead").x

print(lossfunc2(params1))

l2 = -lossfunc2(params2)
params_null = np.concatenate([params, [np.log10(masses.max()), -30.0, 3.0, -3]])
l1 = -lossfunc2(params_null)
likelihood_ratio = l1 - l2
AIC1 = 2 * 3 - 2 * l1
AIC2 = 2 * 7 - 2 * l2
delta_AIC = AIC2 - AIC1
print(delta_AIC, l1, l2)
mgrid = np.logspace(-3, 4, 100001)
logm = np.log10(mgrid)
from imf_components import chabrier_smooth_lognormal_peak_imf

counts, bins = plt.hist(np.log10(masses), 31, histtype="step", edgecolor="black")[:2]
imf_to_bins = (bins.max() - bins.min()) / len(bins) * len(masses)

flat_samples = np.load("chabrier_samples.npy")
for i, p in enumerate(flat_samples[-100:]):
    imf = chabrier_smooth_imf(logm, (p if i > 0 else np.median(flat_samples, axis=0)))
    plt.plot(
        np.log10(mgrid),
        imf * imf_to_bins,
        color="grey",
        label=("Chabrier" if i == 0 else None),
        lw=(0.2 if i > 0 else 1),
        alpha=(0.05 if i > 0 else 1),
    )
    plt.yscale("log")

flat_samples = np.load("chabrier_lognormal_samples.npy")
for i, p in enumerate(flat_samples[-100:]):
    imf = chabrier_smooth_lognormal_peak_imf(logm, (p if i > 0 else np.median(flat_samples, axis=0)))
    plt.plot(
        np.log10(mgrid),
        imf * imf_to_bins,
        color="red",
        label=("Chabrier + Lognormal VMS" if i == 0 else None),
        lw=(0.2 if i > 0 else 1),
        alpha=(0.05 if i > 0 else 1),
    )
    plt.yscale("log")

# imf = chabrier_smooth_lognormal_peak_imf(logm, params_null)
# plt.plot(np.log10(mgrid) , imf * imf_to_bins,color='black',ls='dotted',label="Chabrier-like"); plt.yscale('log')

plt.xlabel(r"$\log M_{\rm \star}$")
plt.ylabel(r"$N$")
plt.ylim(0.5, 5e2)
plt.xlim(-2, 3.5)
plt.legend(loc=1)
plt.savefig("IMF_chabrier_lognormal.png", dpi=300, bbox_inches="tight")

971.4594950485255
-76.86728754736737 -959.6007944115908 -917.1671506379071


In [25]:
# plt.close()
plt.hist(np.e ** flat_samples[:, 1])

(array([  2.,   3.,  33., 111., 210., 276., 210., 102.,  43.,  10.]),
 array([0.29603477, 0.31575078, 0.33546679, 0.35518279, 0.3748988 ,
        0.3946148 , 0.41433081, 0.43404682, 0.45376282, 0.47347883,
        0.49319483]),
 <BarContainer object of 10 artists>)